In [1]:
import pandas as pd
import os,shutil
from datetime import datetime, timedelta
import datetime

In [2]:
workbook = pd.ExcelFile('latest_sample_pinged_data.xlsx')

In [3]:
data = pd.read_excel(workbook)

In [4]:
len(data)

57875

In [5]:
#extracts the filtered data to its own variable
def sheet(source,column, string):
    x = source[source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x
#removes the filtered data from the main variable
def sheet_x(source,column, string):
    x = source[~source[column].str.contains(string,na=False)]
    x = x.sort_values('IP')
    return x

In [6]:
#change last scanned date to_date
data['Last Scanned Date']=pd.to_datetime(data['Last Scanned Date'], format= '%Y-%m-%d').dt.date

In [7]:
#getting duplicates by date
duplicates = data[data.sort_values('Last Scanned Date').duplicated('IP', keep='first')]

<ipython-input-7-6b0a7d62c223>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  duplicates = data[data.sort_values('Last Scanned Date').duplicated('IP', keep='first')]


In [8]:
#dropping duplicates keeping last/recent instance
data = data.sort_values('Last Scanned Date').drop_duplicates('IP', keep='last')

In [9]:
#older than 30days from last scanned date
old = data[data['Last Scanned Date'] < datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [10]:
#30 days from last scanned date
data = data[data['Last Scanned Date'] > datetime.date(2021,1,19)- pd.to_timedelta("30day") ].sort_values('Last Scanned Date' ,ascending=False)

In [11]:
data[(data['IP in CMDB']=='IP in CMDB') & (data['DNS in CMDB'] == 'DNS in CMDB')]

,IP,DNS,Tracking Method,Last Scanned Date,Tags,From,IP in CMDB,DNS in CMDB,VM host URL,Ping,CNAME
2662,10.16.8.77,elkcom-dms004.prod.ams.netledger.com,IP,2021-01-19,all_internal|BU:NSCore|Security|am3_prod_java_...,Cloud,IP in CMDB,DNS in CMDB,NaN,DOWN,elkcom-dms004.prod.ams.netledger.com.
2663,10.16.8.84,backend021.prod.ams.netledger.com,IP,2021-01-19,all_internal|BU:NSCore|Security|am3_prod_java_...,Cloud,IP in CMDB,DNS in CMDB,NaN,DOWN,backend021.prod.ams.netledger.com.
2664,10.16.9.70,cas005.st.ams.netledger.com,IP,2021-01-19,all_internal|BU:NSCore|Security|am3_prod_java_...,Cloud,IP in CMDB,DNS in CMDB,NaN,DOWN,cas005.st.ams.netledger.com.
2665,10.16.9.83,kafka005.prod.ams.netledger.com,IP,2021-01-19,all_internal|BU:NSCore|Security|am3_prod_java_...,Cloud,IP in CMDB,DNS in CMDB,NaN,DOWN,kafka005.prod.ams.netledger.com.
2668,10.16.10.72,acct054.prod.ams.netledger.com,IP,2021-01-19,all_internal|BU:NSCore|Security|am3_prod_java_...,Cloud,IP in CMDB,DNS in CMDB,NaN,DOWN,acct054.prod.ams.netledger.com.
...,...,...,...,...,...,...,...,...,...,...,...
16424,10.10.10.72,shop024.prod.bos.netledger.com,IP,2021-01-08,BU:NSCore|AG:IL_automation|AG:IL_app_NetSuite|...,PCP,IP in CMDB,DNS in CMDB,NaN,DOWN,shop024.prod.bos.netledger.com.
23991,10.8.0.76,ch1-ls0312.chi.netledger.com,IP,2021-01-08,BU:NSCore|BU:NetOps|AG:IL_automation|AG:IL_env...,PCP,IP in CMDB,DNS in CMDB,NaN,DOWN,AVAILABLE-10.8.0.76.ips.chi.netledger.com.
23990,10.8.0.75,ch1-ls0311.chi.netledger.com,IP,2021-01-08,BU:NSCore|BU:NetOps|AG:IL_automation|AG:IL_env...,PCP,IP in CMDB,DNS in CMDB,NaN,DOWN,AVAILABLE-10.8.0.75.ips.chi.netledger.com.
53091,10.18.1.141,db126.prod.dub.netledger.com,IP,2021-01-08,BU:NSCore|AG:IL_automation|AG:IL_app_NetSuite|...,PCP,IP in CMDB,DNS in CMDB,NaN,DOWN,db126.prod.dub.netledger.com.


In [12]:
def cmdb(x):
    ins = x[(x['IP in CMDB']=='IP in CMDB') & (x['DNS in CMDB'] == 'DNS in CMDB')]
    out = x[(x['IP in CMDB'] =='IP NOT in CMDB') | (x['DNS in CMDB'] == 'DNS NOT in CMDB')]
    return ins, out

In [13]:
def writers(filter_frame, filter_title,typed):
    with pd.ExcelWriter(f'{filter_title}/{typed}_{filter_title}.xlsx') as create:
        print(filter_title)
        filter_frame.to_excel(create, index=False)
        
    

In [14]:
def filters (data):
    print('Sorting "Duplicate IP"')
    duplicate = data[data.duplicated(subset=['IP'],keep='last')]
    main = data[~data.duplicated(subset=['IP'],keep='last')]
    print('Sorting "Available"')
    available = sheet(main,'DNS','available')
    main = sheet_x(main,'DNS','available')
    available = available.append(sheet(main,'CNAME','AVAILABLE'))
    main = sheet_x(main,'CNAME','AVAILABLE')
    print('Sorting "VIP DNS"')
    vip = sheet(main,'DNS','vip')
    main = sheet_x(main,'DNS','vip')
    print('Sorting "Internet Facing Assets"')
    pub = sheet(main,'Tags','Internet Facing Assets')
    main = sheet_x(main,'Tags','Internet Facing Assets')
    print('Sorting "Load Balancers"')
    lb = sheet(main,'DNS','-mip|-vlb|-plb')
    main = sheet_x(main,'DNS','-mip|-vlb|-plb')
    #Tags
    lb = lb.append(sheet(main,'Tags','loadbalancer'))
    main = sheet_x(main,'Tags','loadbalancer')
    print('Sorting "Agent"')
    agent = sheet(main,'Tracking Method','AGENT')
    main = sheet_x(main,'Tracking Method','AGENT')
    print('Sorting "Interface"')
    interface = sheet(main,'DNS','interface')
    main = sheet_x(main,'DNS','interface')
    print('Sorting "links"')
    link = sheet(main,'DNS','-link')
    main = sheet_x(main,'DNS','-link')
    print('Sorting "None"')
    blank = sheet(main,'DNS','None|none')
    main = sheet_x(main,'DNS','None|none')
    print('Sorting "DevOps"')
    devops = sheet(main,'Tags','BU:DevOps')
    main = sheet_x(main,'Tags','BU:DevOps')
    print('Sorting "Duplicate DNS"')
    duplicate = duplicate.append(main[main.duplicated(subset=['DNS'],keep='last')])
    main = main[~main.duplicated(subset=['DNS'],keep='first')]
    print('Sorting "LS"')
    ls = sheet(main,'DNS','-ls')
    main = sheet_x(main,'DNS','-ls')
    print('Sorting "CY"')
    cy = sheet(main,'DNS','cy')
    main = sheet_x(main,'DNS','cy')
    print('Sorting "bo1"')
    bo1 = sheet(main,'DNS','BO1|bo1')
    main = sheet_x(main,'DNS','BO1|bo1')
    #Tags
    bo1 = bo1.append(sheet(main,'Tags','BO1|bo1'))
    main = sheet_x(main,'Tags','BO1|bo1')
    print('Sorting "Bronto"')
    bronto = sheet(main,'DNS','Bronto|bronto')
    main = sheet_x(main,'DNS','Bronto|bronto')
    #tags
    bronto = bronto.append(sheet(main,'Tags','Bronto|bronto'))
    main = sheet_x(main,'Tags','Bronto|bronto')
    print('Sorting "OpenAir"')
    openair = sheet(main,'Tags','openair|OpenAir')
    main = sheet_x(main,'Tags','openair|OpenAir')
    print('Sorting "OCI"')
    oci = sheet(main,'DNS','OCI|oci')
    main = sheet_x(main,'DNS','OCI|oci')
    #tags
    oci = oci.append(sheet(main,'Tags','OCI|oci'))
    main = sheet_x(main,'Tags','OCI|oci')
    print('Sorting "Payroll"')
    payroll = sheet(main,'Tags','payroll|Payroll')
    main = sheet_x(main,'Tags','payroll|Payroll')
    print('Sorting "VPN"')
    vpn = sheet(main,'DNS','vpn|ovpn')
    main = sheet_x(main,'DNS','vpn|ovpn')
    #tags
    vpn = vpn.append(sheet(main,'Tags','vpn|ovpn'))
    main = sheet_x(main,'Tags','vpn|ovpn')
    print('Sorting "Console"')
    console = sheet(main,'DNS','console')
    main = sheet_x(main,'DNS','console')
    #tags
    console = console.append(sheet(main,'Tags','console'))
    main = sheet_x(main,'Tags','console')
    print('Sorting "NoUse"')
    nouse = sheet(main,'DNS','nouse')
    main = sheet_x(main,'DNS','nouse')
    print('Sorting "VM"')
    vm = sheet(main,'DNS','snap|\.f\.|\.m\.')
    main = sheet_x(main,'DNS','snap|\.f\.|\.m\.')
    print('Sorting "PDU"')
    pdu = sheet(main,'DNS','pdu-')
    main = sheet_x(main,'DNS','pdu-')
    #IP address filter
    pdu = pdu.append(sheet(main,'IP','^10\..*\..*\.24[0-8]'))
    main = sheet_x(main,'IP','^10\..*\..*\.24[0-8]')
    print('Finished sorting filters')
    filter_frame = [main,lb,devops,link, blank, duplicate, agent, ls, cy, pdu, pub, bo1, bronto, openair, oci, payroll, console, vpn, nouse, vm,interface, available,vip]
    filter_title = ['main','Load Balancer','DevOps','Link', 'None', 'Duplicate', 'Agent', 'LS', 'CY', 'PDU', 'Internet Facing Asset', 'BO1', 'Bronto', 'OpenAir', 'OCI'
                    , 'Payroll', 'Console', 'VPN', 'NoUse', 'VM','Interface', 'Available', 'VIP_NAT']
    for frame, title in zip(filter_frame, filter_title):
#         if len(frame)>0:
#             print(title,len(frame))
#             ins,out = cmdb(frame)
#             if len(ins) >0:
#                 print(f'in_{len(ins)}')
#             else:
#                 pass
            
#             if len(out) >0:
#                 print(f'out_{len(out)}')
#         else:
#             pass
        
        if len(frame) >0:
            if os.path.exists(title):
                shutil.rmtree(title)
                os.mkdir(title)
            else:
                os.mkdir(title)
            ins,out=cmdb(frame)
            if len(ins)>0:
                print (f'{title}_ins,{len(ins)}')
                writers(frame, title,'in_cmdb')
            else:
                pass
            
            if len(out) >0:
                 print (f'{title}_out,{len(out)}')
                 writers(frame, title,'out_cmdb')
            else:
                pass
                

In [15]:
filters(data)

Sorting "Duplicate IP"
Sorting "Available"
Sorting "VIP DNS"
Sorting "Internet Facing Assets"
Sorting "Load Balancers"
Sorting "Agent"
Sorting "Interface"
Sorting "links"
Sorting "None"
Sorting "DevOps"
Sorting "Duplicate DNS"
Sorting "LS"
Sorting "CY"
Sorting "bo1"
Sorting "Bronto"
Sorting "OpenAir"
Sorting "OCI"
Sorting "Payroll"
Sorting "VPN"
Sorting "Console"
Sorting "NoUse"
Sorting "VM"
Sorting "PDU"
Finished sorting filters
main_ins,5705
main
main_out,220
main
Load Balancer_ins,36
Load Balancer
Load Balancer_out,590
Load Balancer
DevOps_out,152
DevOps
Link_out,746
Link
None_out,12325
None
Duplicate_out,4
Duplicate
Agent_ins,2912
Agent
Agent_out,3493
Agent
LS_ins,687
LS
LS_out,56
LS
CY_ins,75
CY
CY_out,201
CY
PDU_ins,1094
PDU
PDU_out,258
PDU
Internet Facing Asset_out,1268
Internet Facing Asset
BO1_ins,3
BO1
BO1_out,3
BO1
Bronto_out,1007
Bronto
OpenAir_out,78
OpenAir
OCI_ins,10
OCI
OCI_out,10
OCI
Console_ins,29
Console
Console_out,4945
Console
VPN_ins,20
VPN
VPN_out,5029
VPN
NoUse_

In [16]:

    
    
        
# with pd.ExcelWriter('test.xlsx') as writer:
#     main.to_excel(writer,sheet_name='main', index=False)
#     agent.to_excel(writer,sheet_name='AGENT', index=False)
#     netops.to_excel(writer,sheet_name='netops', index=False)
#     nscore.to_excel(writer,sheet_name='nscore', index=False)
#     bronto.to_excel(writer,sheet_name='bronto', index=False)
#     openair.to_excel(writer,sheet_name='openair', index=False)
#     payroll.to_excel(writer,sheet_name='payroll', index=False)
#     devops.to_excel(writer,sheet_name='devops', index=False)
#     pub.to_excel(writer,sheet_name='Internet Facing Assets', index=False)
#     nspos.to_excel(writer,sheet_name='NSPOS', index=False)
#     test_asset.to_excel(writer,sheet_name='Test Asset', index=False)
#     legacy.to_excel(writer,sheet_name='LegacyCorp', index=False)
    
    
    
    
    
    
    
    
    